In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz

from RIS_gravity_inversion import regional, synthetic, plotting, optimization
import RIS_gravity_inversion.inversion as inv
import RIS_gravity_inversion.utils as inv_utils
import RIS_gravity_inversion.gravity_processing as grav

import copy
from antarctic_plots import utils, regions, maps, fetch, profile
import zarr
import harmonica as hm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import verde as vd
import scipy as sp
import pvxarray
import rioxarray
import pygmt
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import plotly
import plotly.figure_factory as ff
import seaborn as sns

sns.set_theme()
import optuna
from optuna.storages import JournalStorage, JournalFileStorage
import warnings
import itertools
import pickle
import random


import os

# make beep sound when notify is called at end of long-running cells
from IPython.display import Audio

wave = np.sin(2 * np.pi * 400 * np.arange(10000 * 1) / 10000)
notify = Audio(wave, rate=10000, autoplay=True)

In [ ]:
plot = True
# plot = False
runs = 50

In [ ]:
# choose resolution

layer_spacing = 5e3
# layer_spacing = 10e3

grav_spacing = 5e3
# grav_spacing = 10e3

# Load inversion inputs

In [ ]:
# weights grid and starting bathymetry
starting_prisms = xr.open_zarr(
    f"../inversion_layers/RIS_starting_prisms_{int(layer_spacing)}k_constraints.zarr"
)

# partial bouguer corrected gravity data
df_anomalies = pd.read_csv(
    "../data/RIS_residual_50k_filt_mask.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

# set regions
buffer_region = utils.get_grid_info(starting_prisms.density)[1]
inversion_region = vd.get_region((df_anomalies.easting, df_anomalies.northing))

# set layer parameters
registration = utils.get_grid_info(starting_prisms.density)[4]

print(f"layer spacing: {layer_spacing}m")
print(f"registration: {registration}")
print("inversion region = ", inversion_region)
print("buffer region = ", buffer_region)

if layer_spacing == 5e3:
    fname = "../inversion_layers/bed_from_constraints_5k.zarr"
elif layer_spacing == 10e3:
    fname = "../inversion_layers/bed_from_constraints_10k.zarr"
bed_from_constraints = xr.open_zarr(fname).bed
bed_from_constraints = fetch.resample_grid(
    bed_from_constraints,
    region=buffer_region,
    registration=registration,
    verbose="q",
).rename({"x": "easting", "y": "northing"})

# constraints
if layer_spacing == 5e3:
    constraints_fname = "../data/bed_constraints_5k.csv.gz"
elif layer_spacing == 10e3:
    constraints_fname = "../data/bed_constraints_10k.csv.gz"
else:
    raise ValueError("No constraint point files available at specified spacing!")

constraints = pd.read_csv(
    constraints_fname,
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

# mark points inside grounding line as "inside"
measures_shelves = fetch.measures_boundaries(version="IceShelf")
ice_shelves = gpd.read_file(measures_shelves)
RIS_seperate = ice_shelves[ice_shelves.NAME.isin(["Ross_West", "Ross_East"])]
RIS = RIS_seperate.dissolve()
gdf = gpd.GeoDataFrame(
    constraints,
    geometry=gpd.points_from_xy(x=constraints.easting, y=constraints.northing),
    crs="EPSG:3031",
)
constraints["inside"] = gdf.within(RIS.geometry[0])
constraints.drop(columns="geometry", inplace=True)

# ensure all points are within buffer region
constraints = utils.points_inside_region(
    constraints, buffer_region, names=("easting", "northing")
)
print(f"bathymetry control points:{len(constraints)}")

# surface
fname = "../inversion_layers/bedmachine_surface.zarr"
surface_full = xr.open_zarr(fname).surface
surface = fetch.resample_grid(
    surface_full,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
surface_layer_spacing = fetch.resample_grid(
    surface_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
# icebase
fname = "../inversion_layers/bedmachine_icebase.zarr"
icebase_full = xr.open_zarr(fname).icebase
icebase = fetch.resample_grid(
    icebase_full,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
icebase_layer_spacing = fetch.resample_grid(
    icebase_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)

# bed
bedmachine_bed = xr.open_zarr("../inversion_layers/bedmachine_bed.zarr")["bed"]
bedmachine_bed_layer_spacing = fetch.resample_grid(
    bedmachine_bed,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)

# final inverted bathymetry model
fname = "../results/weights_mask_prism_results.csv.gz"
inverted_bathy = pd.read_csv(
    fname,
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
inverted_bathy = inverted_bathy.set_index(["northing", "easting"]).to_xarray()[
    inverted_bathy.columns[-1]
]

# Add uncertainties 
2.9 mGal for gravity data

5% depth from ice surface for constraints within ice shelf

In [ ]:
# sample ice surface elevation into dataframe
# from bedmachine,
constraints = profile.sample_grids(
    constraints,
    surface,
    "surface_z_ellipsoidal",
    coord_names=["easting", "northing"],
)

# calculate error as 5% of depth from surface
constraints["z_error"] = 10
depth = (
    constraints[constraints.inside].surface_z_ellipsoidal
    - constraints[constraints.inside].upward
)
uncert = 0.05 * depth

constraints.loc[constraints[constraints.inside].index, "z_error"] = uncert

constraints["low_lim"] = constraints.upward - constraints.z_error
constraints["high_lim"] = constraints.upward + constraints.z_error

print(
    f"mean uncertainty inside RIS: {np.nanmean(constraints[constraints.inside].z_error)} m"
)
print(
    f"mean uncertainty outside RIS: {np.nanmean(constraints[constraints.inside==False].z_error)} m"
)
constraints.describe()

In [ ]:
df_anomalies["uncert"] = 2.9
df_anomalies

# Setup Monte-Carlo sampling
* define sampling methods for the uncertainties of both the gravity and constraints data
* run full inversion with sampled gravity
    * including: 
        * corrections for parital bouguer and starting bed
        * DC shift
        * regional / residual seperation
        * inversion
* find standard deviation per grid cell of all resulting inverted bathymetry models

## Run inversion workflow

In [ ]:
bedmachine_df_buffer, _, bedmachine_masked = inv_utils.bedmachine_buffer_points(
    buffer_width=20e3,
    bedmachine_bed=bedmachine_bed_layer_spacing,
    # plot=True,
)

In [ ]:
kwargs = {
    "grav_spacing": grav_spacing,
    "inversion_region": inversion_region,
    "buffer_region": buffer_region,
    "layer_spacing": layer_spacing,
    # optional:
    "weights_grid": starting_prisms.weights,
    "starting_bed_prisms": starting_prisms,
    "starting_bed": bed_from_constraints,
    "icebase_lowres": icebase_layer_spacing.rename({"x": "easting", "y": "northing"}),
    "icebase_fullres": icebase,
    "surface_fullres": surface,
    "bed_buffer_points": bedmachine_df_buffer,
}
inversion_args = {
    "input_grav_column": "Gobs_corr_shift",
    "apply_weights": False,
    # "apply_weights": True,
    "weights_after_solving": True,
    "max_iterations": 20,
    "l2_norm_tolerance": 0.2,
    "delta_l2_norm_tolerance": 0.8,
    "perc_increase_limit": 0.2,
    "upper_confining_layer": icebase_layer_spacing,
    "deriv_type": "annulus",
    "solver_type": "scipy least squares",
}

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_full.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    sample_grav=True,  # will recalculate regional field
    sample_constraints=True,  # will recalculate starting bed, bed gravity, and regional field
    sample_tension_factor=True,  # will recalculate regional field
    sample_ice_density=True,  # will recalculate bouguer anomalies and regional field
    sample_water_density=True,  # will recalculate bouguer anomalies and regional field
    sample_sediment_density=True,  # will recalculate bed gravity and regional field
    sample_inner_bound=True,  # will recalculate weights grid
    sample_outer_bound=True,  # will recalculate weights grid
    sample_solver_damping=True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Full Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_gravity.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    sample_grav=True,  # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Gravity Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_constraints.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    sample_constraints=True,  # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Constraints Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_tension.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    sample_tension_factor=True,  # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Tension factor Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_ice_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    sample_ice_density=True,  # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Density (ice) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_water_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    sample_water_density=True,  # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Density (water) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_sediment_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    sample_sediment_density=True,  # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Density (sediment) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_inner_bound_density.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    sample_inner_bound=True,  # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Bounds (inner) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_outer_bound.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    sample_outer_bound=True,  # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Bounds (outer) Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_solver_damping.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    sample_solver_damping=True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Damping Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_data.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    sample_grav=True,  # will recalculate regional field
    sample_constraints=True,  # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Data Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_densities.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    sample_ice_density=True,  # will recalculate bouguer anomalies and regional field
    sample_water_density=True,  # will recalculate bouguer anomalies and regional field
    sample_sediment_density=True,  # will recalculate bed gravity and regional field
    # sample_inner_bound = True,      # will recalculate weights grid
    # sample_outer_bound = True,      # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Densities Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
fname = "../data/monte_carlo_results/monte_carlo_bounds.zarr"

topos, params, values, grav_dfs = inv_utils.monte_carlo_inversion_uncertainty_loop(
    fname=fname,
    runs=runs,
    grav=df_anomalies,
    constraints=constraints,
    # sample_grav = True,             # will recalculate regional field
    # sample_constraints = True,      # will recalculate starting bed, bed gravity, and regional field
    # sample_tension_factor = True,   # will recalculate regional field
    # sample_ice_density = True,      # will recalculate bouguer anomalies and regional field
    # sample_water_density = True,    # will recalculate bouguer anomalies and regional field
    # sample_sediment_density = True, # will recalculate bed gravity and regional field
    sample_inner_bound=True,  # will recalculate weights grid
    sample_outer_bound=True,  # will recalculate weights grid
    # sample_solver_damping = True,
    inversion_args=inversion_args,
    plot=plot,
    mask="../plotting/RIS_outline.shp",
    title="Bounds Monte Carlo simulation",
    **kwargs,
)
print(params[-1])
print(values[-1])

In [ ]:
# residuals = [utils.RMSE(df[list(df.columns)[-1]]) for df in grav_dfs]
# stats = inv_utils.model_ensemble_stats(topos, weights = residuals)
# z_stdev, weighted_stdev = stats[2], stats[4]
# grd = z_stdev - weighted_stdev
# grd = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp",
#                  xr_grid = grd, masked=True, invert=False)
# maps.plot_grd(
#     grd,
#     cmap="inferno",
#     reverse_cpt=True,
#     robust=True,
#     hist=True,
#     cbar_label="weighted standard deviation (m)",
#     # title=title,
# )

## Calculate standard deviation
* calculate standard deviation of each grid cell for the entire suite of invert bathymetries

In [ ]:
# values_df = pd.DataFrame(values)
# fig, axes = plt.subplots(1,3)
# sns.histplot(ax=axes[0], data=values_df, x="ice_density", kde=True)
# sns.histplot(ax=axes[1], data=values_df, x="water_density", kde=True)
# sns.histplot(ax=axes[2], data=values_df, x="sediment_density", kde=True)

In [ ]:
# grd = ds_merged['run_num'].std("runs").rename("z_std")
# grd = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = grd, masked=True, invert=False)

# fig = maps.plot_grd(
#     grd,
#     cmap="inferno",
#     reverse_cpt=True,
#     robust=True,
#     hist=True,
#     cbar_label="standard deviation (m)",
#     title='Combined Monte Carlo uncertainty',
# )

# fig.plot(
#     x=constraints.easting,
#     y=constraints.northing,
#     fill="black",
#     style="c.01c",
# )
# fig.plot(
#     x=constraints[constraints.inside].easting,
#     y=constraints[constraints.inside].northing,
#     fill="white",
#     pen=".3p,black",
#     style="c.08c",
#     label="Bed constraints",
# )
# fig.legend()

# fig.show()

In [ ]:
# grd = ds_merged['run_num'].median("runs").rename("z_median")
# grd = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = grd, masked=True, invert=False)

# fig = maps.plot_grd(
#     grd,
#     cmap="inferno",
#     reverse_cpt=True,
#     robust=True,
#     hist=True,
#     cbar_label="median (m)",
#     title='Combined Monte Carlo median',
# )

# fig.plot(
#     x=constraints.easting,
#     y=constraints.northing,
#     fill="black",
#     style="c.01c",
# )
# fig.plot(
#     x=constraints[constraints.inside].easting,
#     y=constraints[constraints.inside].northing,
#     fill="white",
#     pen=".3p,black",
#     style="c.08c",
#     label="Bed constraints",
# )
# fig.legend()

# fig.show()

In [ ]:
# merged_min = ds_merged['run_num'].min("runs")
# merged_max = ds_merged['run_num'].max("runs")
# merged_min = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = merged_min, masked=True, invert=False)
# merged_max = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = merged_max, masked=True, invert=False)

# _ = utils.grd_compare(
#     merged_min,
#     merged_max,
#     plot=True,
#     grid1_name="Min",
#     grid2_name="Max",
#     title='Combined Monte Carlo analysis',
#     cbar_label="variance (m)",
#     diff_cmap="inferno",
#     diff_maxabs=False,
#     cmap="rain",
#     reverse_cpt=True,
#     rmse_in_title=False,
#     subplot_labels=True,
#     robust=True,
#     hist=True,
#     # shp_mask="../plotting/RIS_outline.shp",
# )

In [ ]:
# inverted_bathy_mask = utils.mask_from_shp(shapefile="../plotting/RIS_outline.shp", xr_grid = inverted_bathy, masked=True, invert=False)
# max_to_final = merged_max - inverted_bathy_mask
# min_to_final = merged_min - inverted_bathy_mask

# fig = maps.plot_grd(
#     max_to_final,
#     title="Max variance - final model",
#     cmap="vik+h0",
#     robust=True,
#     hist=True,
# )
# fig = maps.plot_grd(
#     min_to_final,
#     title="Min variance - final model",
#     cmap="vik+h0",
#     robust=True,
#     hist=True,
#     fig = fig,
#     origin_shift = "xshift",
# )
# fig.show()